In [134]:
import csv
import pandas as pd
import numpy as np

import folium
import folium.plugins as plugins
import ipywidgets as widgets

from IPython.display import HTML
from IPython.display import display
from IPython.display import Image

#parse and load csv file into a Pandas DataFrame
csvfile = 'https://raw.githubusercontent.com/rkalyanapurdue/smolensk/master/Division.csv'
df = pd.read_csv(csvfile,parse_dates=['MAP_DATE'])

#group the data by Army group and num
grouped_div = df.groupby(['Army_Group','Num_Name'])

division_map = folium.Map([54.78, 32.04],zoom_start=6)

features= []

for key, item in grouped_div:
    
    div_df = grouped_div.get_group(key)[['POINT_Y','POINT_X','OBJECTID','MAP_DATE', 'Type_D']]
    
    div_data = dict()
    
    # get Type_D value
    name = grouped_div.get_group(key).Type_D.to_string()
    # split into number (arr[0]) and name (arr[1])
    arr = name.split()
    name = arr[1]
    
    # get Type_ND value
    subname = grouped_div.get_group(key).Type_ND.to_string()
    
    # split into number (arr[0]) and name (arr[1])
    arr = subname.split()
    
    # Do various sorts of text processing necessary to obtain subname in desired format
    
    # remove the arr[0]
    del arr[0]
    
    # remove all elements at indices >= 2
    del arr[2:len(arr)]
    
    # remove all elements with numbers in them
    arr = [x for x in arr if not (x.isdigit() or x[0] == '-' and x[1:].isdigit())]
    
    # remove all elements equal to 'NaN'
    arr = [x for x in arr if x != 'NaN']
    
    # remove all elements equal to '<Null>'
    arr = [x for x in arr if x != '<Null>']
    
    # remove all elements equal to 'ELE'
    arr = [x for x in arr if x != 'ELE']
    
    # remove all elements equal to 'MIN'
    arr = [x for x in arr if x != 'MIN']
    
    subname = ' '.join(arr)
    #concatenate arr[1], arr[2] (if existent)

    #print(subname)
    
    # set calculatedUrl based on div_df attribute 'Type_D'
    # NOTE - the CSV file is not quite organized; some Type_ND attributes don't match Type_D, etc
    # I would need to match by both Type_ND and Type_D for perfect icon assignment
    # let name = attr Type_D instance val
    
    if name == 'INF':   #infantry
        calculatedUrl = 'https://user-images.githubusercontent.com/16271606/50296442-83fc0980-0448-11e9-8255-7bf42499ea9c.png'
        
    elif name == 'SEC': # section?
        calculatedUrl = 'https://user-images.githubusercontent.com/16271606/50296448-852d3680-0448-11e9-9f80-c09c248dd0d2.png'
        
    elif name == 'PZR': # Panzer (tank)
        calculatedUrl = 'https://user-images.githubusercontent.com/16271606/50296447-8494a000-0448-11e9-82a7-66cc64955626.png'
        
    elif name == 'MOT': # Motor brigade
        if subname == 'MOT BDE': # MOT BDE
            calculatedUrl = 'https://user-images.githubusercontent.com/16271606/50296443-83fc0980-0448-11e9-86a4-d7db148a9c1e.png'
    
        elif subname == 'MOT RGT': # MOT REG
            calculatedUrl = 'https://user-images.githubusercontent.com/16271606/50296444-83fc0980-0448-11e9-8e96-e1a88581e486.png'
    
        elif subname == '': # MOT
            calculatedUrl = '(https://user-images.githubusercontent.com/16271606/50296445-8494a000-0448-11e9-8c7f-f8fe90a86442.png'
        
        else:   # default: Type_D attribute instance val is null
                # 60 x 60 red square used to indicate null/mispelled/etc Type_D value
            calculatedUrl = 'https://user-images.githubusercontent.com/16271606/50376693-9314c000-05de-11e9-911d-de9edff7b911.png'
            print("Name:" + name)
            print("Subname: " + subname)
        
        
    elif name == 'CAV': # Cavalry
        calculatedUrl = 'https://user-images.githubusercontent.com/16271606/50296440-83637300-0448-11e9-9201-9ee5c42b6bb6.png'
        
    elif subname == 'FWD DET': # Fwd Det
        calculatedUrl = 'https://user-images.githubusercontent.com/16271606/50296441-83637300-0448-11e9-8e9b-bc0d940f8aee.png'    
        
    else: # default: Type_D attribute instance val is null
        if subname == 'MOT BDE': # MOT BDE
            calculatedUrl = 'https://user-images.githubusercontent.com/16271606/50296443-83fc0980-0448-11e9-86a4-d7db148a9c1e.png'
            
        elif subname == 'MOT RGT': # MOT REG
            calculatedUrl = 'https://user-images.githubusercontent.com/16271606/50296444-83fc0980-0448-11e9-8e96-e1a88581e486.png'    
        
        else:   # default: Type_D attribute instance val is null
            # 60 x 60 red square used to indicate null/mispelled/etc Type_D value
            calculatedUrl = 'https://user-images.githubusercontent.com/16271606/50376693-9314c000-05de-11e9-911d-de9edff7b911.png'
            print("Name:" + name)
            print("Subname: " + subname)
    
    for row in div_df.itertuples():
        if row.MAP_DATE not in div_data:
            div_data[row.MAP_DATE] = dict()
            div_data[row.MAP_DATE]['POINT_Y'] = row.POINT_Y
            div_data[row.MAP_DATE]['POINT_X'] = row.POINT_X
    div_locs = []
    for key in sorted(div_data):
        div_loc = dict()
        div_loc['MAP_DATE'] = key
        div_loc['POINT_Y'] = div_data[key]['POINT_Y']
        div_loc['POINT_X'] = div_data[key]['POINT_X']
        div_locs.append(div_loc)

    for i in range(0,len(div_locs)-1):
        row1 = div_locs[i]
        row2 = div_locs[i+1]
        
        data = {
                  "type": "Feature",
                  "geometry": {
                      "type": "LineString",
                      "coordinates": [[row1['POINT_X'],row1['POINT_Y']],[row2['POINT_X'],row2['POINT_Y']]]
                  },
                  "properties":{
                      "icon": "marker", 
                      "iconstyle": {
                          "iconSize": [30, 30], 
                          #"iconUrl":"https://github.com/rkalyanapurdue/smolensk/raw/master/tank.png"},
                          #"iconUrl":"https://user-images.githubusercontent.com/16271606/50296448-852d3680-0448-11e9-9f80-c09c248dd0d2.png"},
                          "iconUrl": calculatedUrl},
                      "times": [str(row1['MAP_DATE'])[:10],str(row2['MAP_DATE'])[:10]]}
                }
        features.append(data)
        
plugins.TimestampedGeoJson({
   'type': 'FeatureCollection',
   'features': features
}, add_last_point=True).add_to(division_map)
    
#division_map
division_map.save('map.html')